# boto3 library


`boto3` is the Python SDK for AWS (Amazon Web Service).  
It is the python library used to interact with all AWS services (S3, EC2, DynamoDB...). 

This SDK contains an extremely large number of methods, all with a lot of options, across all available AWS services.   
This tutorial just shows the main concepts and code samples for a few actions.  
For a full guide on this library, refer to the official AWS documentation : https://boto3.amazonaws.com/v1/documentation/api/latest/index.html


### Setup

To use the AWS SDK, we need to be authenticated in an AWS account.  
- create an AWS account if needed
- install the AWS CLI locally
- run `aws configure` from a local terminal to save the AWS account access/secret keys under `~/.aws/credentials` and the default region under `~/.aws/config`

Alternatively, you can just open an AWS CloudShell from the AWS console and use `boto3` from a python script inside it.  
The CloudAWS Shell comes with a pre-installed AWS CLI and inherits the credentials from the logged user by default.


### Concepts of the boto3 SDK

A **client** is a low-level interface to an AWS service offering methods for each possible API call.  
This is the original way to call the AWS API from Python.

A **resource** is an object-oriented interface to an AWS service providing a higher level of abstraction.  
The resource has a number of attributes and methods specific to the service they apply to.  
The resource API is newer and does not support yet all possible API calls.


### Example with an S3 client

The below code creates a new bucket in S3, adds a file in it, then remove the file and delete the bucket using the `client` interface.

In [82]:
import boto3
import time


# Create an S3 client
s3 = boto3.client('s3')

# Iterable list of buckets in this account
s3.list_buckets()['Buckets']

# Create a new bucket
bucket_name = 'test-bucket-' + str(int(time.time()))   # globally unique bucket name
location    = {'LocationConstraint': 'ap-northeast-1'}
s3.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration=location)

# Create a file in this bucket
file_name = 'test.txt'
s3.put_object(
    Body=file_name,
    Bucket=bucket_name,
    Key=file_name)

# Delete the file in this bucket
s3.delete_object(Bucket=bucket_name, Key=file_name)

# Delete the bucket
s3.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'VD9E8W0N3Q1ZQXBD',
  'HostId': '5Rgg3IWS+IDSY3L3AbytM/vlZsp0XEN/eIlpdHsQgfu5h/Rub8lQI52LLMaISxJFrx+8C6Actqs=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '5Rgg3IWS+IDSY3L3AbytM/vlZsp0XEN/eIlpdHsQgfu5h/Rub8lQI52LLMaISxJFrx+8C6Actqs=',
   'x-amz-request-id': 'VD9E8W0N3Q1ZQXBD',
   'date': 'Sat, 03 Jul 2021 15:11:21 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Example with an S3 resource

The below code performs the exact same actions as above, but using a `resource` interface instead of a `client` :

In [83]:
# Create an S3 resource
s3 = boto3.resource('s3')

# Iterable list of buckets
# With the resource interface we have a "buckets" attribute wrapping the API call
s3.buckets.all()

# Create a new bucket
# With the resource interface, we can instanciate a new Bucket object
bucket_name = 'test-bucket-' + str(int(time.time()))   # globally unique bucket name
location    = {'LocationConstraint': 'ap-northeast-1'}
s3.Bucket(name=bucket_name).create(CreateBucketConfiguration=location)

# Create a file in this bucket    
file_name = 'test.txt'
s3.Object(bucket_name, file_name).put(Body=open(file_name, 'rb'))

# Delete the file in this bucket
s3.Bucket(name=bucket_name).delete_objects(
    Delete={
        'Objects': [
            {'Key': file_name}
        ]
    })

# Delete this bucket
s3.Bucket(name=bucket_name).delete()

{'ResponseMetadata': {'RequestId': 'X7HFCTYPDWGRGH12',
  'HostId': 'ADRc0tXtsJd/6qXZ4Evq8qEdEhX7UpUSY7GaAbyYPtDxrZbUy0lwnCwUnZCEaviC8u3pssi13cM=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'ADRc0tXtsJd/6qXZ4Evq8qEdEhX7UpUSY7GaAbyYPtDxrZbUy0lwnCwUnZCEaviC8u3pssi13cM=',
   'x-amz-request-id': 'X7HFCTYPDWGRGH12',
   'date': 'Sat, 03 Jul 2021 15:11:23 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Sample SQS code

The below code creates a queue in AWS SQS (Simple QUeue Service), sends a message to it, polls and deletes the message, and finally deletes the queue.

In [126]:
# Create the client
sqs = boto3.client('sqs')

# Create a queue
queue_name = 'boto3-test-queue-' + str(int(time.time()))
queue      = sqs.create_queue(QueueName=queue_name)
queue_url  = queue['QueueUrl']

# Poll a message from the queue
response = sqs.receive_message(QueueUrl=queue_url)
assert 'Messages' not in response     # No actual message returned since the queue is empty

# Send a message to the queue
sqs.send_message(
    QueueUrl=queue_url,
    MessageAttributes={
        'Title': {
            'DataType': 'String',
            'StringValue': 'The Three-Body Problem'
        },
        'Author': {
            'DataType': 'String',
            'StringValue': 'Liu Cixin'
        }
    },
    MessageBody='Book Suggestion'
)

# Receive a message
response = sqs.receive_message(
    QueueUrl=queue_url,
    MaxNumberOfMessages=1,
    MessageAttributeNames=['All'],   # By default, do not fetch the message attribute
    VisibilityTimeout=10)

assert 'Messages' in response
msg = response['Messages'][0]
assert msg['Body'] == 'Book Suggestion'
assert msg['MessageAttributes']['Author']['StringValue'] == 'Liu Cixin'

# Delete the message from the queue (so it cannot be polled again)
receipt = msg['ReceiptHandle']
print(receipt)
sqs.delete_message(
    QueueUrl=queue_url,
    ReceiptHandle=receipt
)

# Delete the queue
sqs.delete_queue(QueueUrl=queue['QueueUrl'])

AQEBiFY4PxJ4dcvkhihKHO6J6OMYVQgu8iNQP19VAe1DFAXHTtnp/pVmtIQvwKUrWmUnKqRhMVV9mcAD1yiPVfDxWlv90G9bjCG8fqZ3nQJfbwU+OK77oaNmXeRy/kihnvAdF5ehxgTG0KZ83y2+D63YHEfrziJDjDe8ifHV6/nYr7zH+lyfFtJSww/2/398kK/i/+UkhpjdvtnprmE+S+mqZZJRWkzLb3qNQ+fqhB3Y/YXV7yEBJSXZDdqKzQfd8Bwyq/Vw0FFIwdoYzGy29bEK/v1ID5DPg8L97Mf7ROOM92MMpV+ZtjwNorMgp6rX1P8oDZuSIhj7MHELMCfQL2tcmUdlyPRojlTrIpyqRekBf+TDXD1uEIa/Xa0ydqzdUy9I10Lt+ODEfpfRwGFbZhl/7/yb02Sz2lYdnCX1cH3rxao=


{'ResponseMetadata': {'RequestId': '4147a085-a0dc-5925-9807-f9de8593340c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4147a085-a0dc-5925-9807-f9de8593340c',
   'date': 'Sun, 04 Jul 2021 02:55:11 GMT',
   'content-type': 'text/xml',
   'content-length': '211'},
  'RetryAttempts': 0}}